# Finding QSOs

In this notebook we will attempt to mine for [Quasi-Stellar Objects](https://en.wikipedia.org/wiki/Quasar)
(aka Quasars) in photometric catalogues. The objects mimic stars, since they are point-like, but are actually
extra-galactic objects. Gaia provides equisite proper motion constraints and QSOs should appear as stationary objects.
A cut on having ~zero proper motion is thus expected to be a powerful method to initially find QSO candidates.

This notebook follows in part the methodology of [Heintz et al. 2018](https://arxiv.org/pdf/1805.03394.pdf), with
additional scope by further investigating optical colours of QSOs and building a Random Forest classifier to find new
QSO candidates.

The basic workflow is:

1. Perform a high-galactic latitude 1-degree cone search in Gaia
2. Calculate quantites and perform cuts on the data directly in Python
3. Cross-match Gaia sources with SDSS photometry
4. Find spectroscopically confirmed QSOs within the cross-matched sources
5. Investigate features of QSOs compared to other field sources to find QSO candidates
6. Train a Random Forest classifier to classify sources as QSOs based on proper-motion and colour

Throughout this we will employ data-mining skills such as retrieving, exploring and analysing data efficiently, as well
as employing simple machine learning techniques to our findings.

There are example exercises contained within this notebook that you can choose from (or use as inspiration) to fulfill
the assignment if you are required to do so for MPAGS credit.

### Preamble

Imports etc.

In [ ]:
import warnings

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import table
from astroquery.gaia import Gaia
from astroquery.sdss import SDSS

warnings.filterwarnings('ignore')  # supress some numpy warnings about converting masked elements

plt.style.use("seaborn-darkgrid")
plt.rcParams['figure.figsize'] = [12, 8]

## 1. Perform a high-galactic latitude 1-degree cone search in Gaia

We will use [Astroquery's Gaia interface](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)
to programmatically query for sources in a 1-degree cone at a high galactic latitude.

First define our cone centre. You can change the ra and dec values to make your own unique results, but
keep within the [SDSS footprint](https://classic.sdss.org/dr7/coverage/) and at high galactic latitude - shifting
either anywhere within +/-20 degrees is safe.

In [ ]:
# Try changing ra and dec to get your own results
ra_cone = 180 * u.degree
dec_cone = 30 * u.degree

# These are used to define our cone centre, and we set the radius
coord = SkyCoord(ra=ra_cone, dec=dec_cone, frame='icrs')
radius = 1 * u.degree

In [ ]:
# Perform a Gaia cone search and get results
Gaia.ROW_LIMIT = -1  # do not limit the number of results returned (default is 50 otherwise)
job = Gaia.cone_search_async(coord, radius)  # use async if retrieving large results (>2000 rows)
gaia_results = job.get_results()
gaia_results  # the notebook will display the table in a nice format if we end the cell with it

Our results are stored as an [Astropy Table](https://docs.astropy.org/en/stable/table/).
Let's check what data we have in our table:


In [ ]:
gaia_results.info

In order to further understand what the columns mean, we should always refer to the **schema** documentation
for the database, which describes the data and its structure. For example, for the main `gaia_source`
table we queried above, we can refer to
[this page](https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html).

## 2. Calculate quantites and perform cuts on the data directly in Python

Our data are held as an [Astropy Table](https://docs.astropy.org/en/stable/table/) - this allows a lot of quick
numpy-like (array-based) analysis, with the benefit of a human-readable structure (column names etc.).

We will investigate aspects of the data and to show examples of **array-based** operations and filtering data.

### Calculate total proper motion of sources

In our hyopthesis, a selection for sources consistent with zero proper motion should be a good first cut to find QSOs.
Let's calculate the total proper motion (as the proper motion in ra and dec added in quadrature),
and along the way demonstrate why we don't use loops whenever possible:

In [ ]:
def get_pmtot_loop(tbl):
    """Don't do this!"""
    pmtot = []
    for row in tbl:
        pmtot.append((row["pmra"]**2 + row["pmdec"]**2)**0.5)
    return pmtot

print("Timing for calculating pmtot with for loop:")
%time pmtot_loop = get_pmtot_loop(gaia_results)

def get_pmtot(tbl):
    """Do this!"""
    return (tbl["pmra"]**2 + tbl["pmdec"]**2)**0.5

print("\nTiming for calculating pmtot with array operation:")
%time pmtot = get_pmtot(gaia_results)

assert np.all(pmtot_loop == pmtot), "results don't match"
# this would raise an error if methods gave different results
# (there is a slight detail in that the for loop returns a list, whereas the array operation returns a column
# - practically they are the same results, but the column has additional astropy.table.Table methods.)

A factor of ~hundreds speed up for this example!

We can now assign these calculated total proper motion values to our main results.
We also populate the corresponding proper motion uncertainty (taking care of the correlation coefficient), as well
as the significance of the proper motion (`=pm/sigma_pm`)

In [ ]:
gaia_results["pmtot"] = pmtot
gaia_results["pmtot_error"] = (
                        (gaia_results["pmra_error"]**2 + gaia_results["pmdec_error"]**2
                        + 2 * gaia_results["pmra_pmdec_corr"] * gaia_results["pmra_error"] * gaia_results["pmdec_error"]
                            )** 0.5)
gaia_results["pm_significance"] = gaia_results["pmtot"]/gaia_results["pmtot_error"]

### Filter the sources based on magnitude

Our aim is to find QSO candidates. We have astrophysical reasons to expect these are going to be relatively
faint (cf. foreground Galactic stars, which are at a full range of brightness). Because of this, we
will apply a simple cut on apparant magnitude to limit things at the faint end of Gaia's detection distribution.

To decide, let's plot the apparant magnitude distribution in Gaia's `G` band.

In [ ]:
plt.hist(gaia_results["phot_g_mean_mag"], bins=np.arange(10, 23, 0.5))
plt.xlabel("G [mag]")
plt.ylabel("N")

We can see that Gaia detection efficiency falls off rapidly around 21 mag. We set our lower cut to 20 mag because we
want reasonably robust photometry (and require that there is a full proper motion solution). Bright sources are very
likely to be foreground stars, so we also cut anything brighter than 18 mag.

In [ ]:
# Produce a boolean mask used to select rows meeting our criteria
mag_mask = (gaia_results["phot_g_mean_mag"] > 18) & (gaia_results["phot_g_mean_mag"] < 20)
print(f"mag_mask = {mag_mask}")
print(f"There are {np.sum(mag_mask)} sources that pass magnitude cuts")
gaia_results_faint = gaia_results[mag_mask]

(In reality you would perform such cuts prior to calculating any quantities on the data, for efficiency.)


## 3. Cross-match Gaia sources with SDSS photometry

Gaia contains pre-baked cross-matches to external catalogues we could use, but instead we will perform manual
cross-matching in Python to demonstrate it.


### Query SDSS for sources

Firstly, we need to grab SDSS sources in our cone. The Astroquery SDSS interface is a bit different from the Gaia one,
unfortunately, and we will also need to specify the columns we want explicitly. Again, we'll refer to the
[schema](https://skyserver.sdss.org/dr14/en/help/browser/browser.aspx#&&history=description+PhotoObjAll+U) to find the
columns we want - we're interested in the position, type, and psf photometry of each source

In [ ]:
photoobj_fields=["ra", "dec", "type", "psfMag_u", "psfMag_g", "psfMag_r", "psfMag_i", "psfMag_z"]
region_radius = radius / np.cos(np.deg2rad(coord.dec.degree))  # this is a correction since query_region doesn't account for cos(dec) factor in ra.
sdss_results = SDSS.query_region(coord, region_radius, photoobj_fields=photoobj_fields)
# immediately cut out any that are not of type=6 (i.e. star)
sdss_results = sdss_results[sdss_results["type"] == 6]
sdss_results


### Cross match Gaia and SDSS

To perform a cross-match, we will use
[Astropy's catalogue matching](https://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs) tools.
For this we need [`SkyCoord`](https://docs.astropy.org/en/stable/coordinates/) objects of our positions in each
catalogue before performing sky-matching

In [ ]:
gaia_coord = SkyCoord(gaia_results_faint["ra"], gaia_results_faint["dec"], unit=u.degree)
sdss_coord = SkyCoord(sdss_results["ra"], sdss_results["dec"], unit=u.degree)
idx, d2d, _ = gaia_coord.match_to_catalog_sky(sdss_coord)

`idx` is now an array of indices in `sdss_coord` that are closest to each `gaia_coord`. We can use `d2d`, the 2D (on-sky)
distance to make a cut on the matches to be within 1 arcsecond.


In [ ]:
close_match = d2d < 1 * u.arcsec  # find where the separations are within our threshold
print(f"Found {np.sum(close_match)} close matches")

# limit our gaia results to those with a close match
gaia_matched = gaia_results_faint[close_match]
# select the close matched SDSS results
sdss_matched = sdss_results[idx[close_match]]

# a boolean mask denoting no proper motion matches
no_pm_mask = gaia_matched["pm_significance"] < 2

As a sanity check, plot the positions of the matched sources, as well as the no proper motion subset.

In [ ]:
plt.scatter(sdss_matched["ra"], sdss_matched["dec"], s=20, facecolor=None, label="Matched SDSS sources")
plt.scatter(gaia_matched["ra"], gaia_matched["dec"], s=5, label="Matched Gaia sources (18 < G < 20)")
plt.scatter(gaia_matched[no_pm_mask]["ra"], gaia_matched[no_pm_mask]["dec"], s=100, c="None", marker="s", edgecolors="C3", linewidths=2, label="No PM sources")
plt.legend(loc="upper left")
plt.xlabel("RA [deg]")
plt.ylabel("Dec [deg]")

## 4. Find spectroscopically confirmed QSOs within the cross-matched sources

We can exploit spectroscopic datasets to find known QSOs in our cone. For this we will appeal to SDSS spectra. (Another
general extragalactic tool to use could be the [NASA Extragalactic Database](https://ned.ipac.caltech.edu/).)

Here we want sources with a spectrum associated (`spectro=True`), and we define key columns to return, again referring to
the appropriate [schema](https://skyserver.sdss.org/dr14/en/help/browser/browser.aspx#&&history=description+SpecObjAll+U)
information.

*(The `sciencePrimary` is used since we query all spectra, and we only want the classification from the best
science spectrum at this location. Otherwise some objects can have conflicting spectral classes.)*

In [ ]:
specobj_fields=["ra", "dec", "class", "sciencePrimary"]
sdss_spec_results = SDSS.query_region(coord, region_radius, spectro=True, specobj_fields=specobj_fields)
# immediately cut out any that are not sciencePrimary spectra
sdss_spec_results = sdss_spec_results[sdss_spec_results["sciencePrimary"] == 1]
qso_mask = sdss_spec_results["class"] == "QSO"  # a boolean mask denoting QSO spectra
print(f"Found {len(sdss_spec_results)} spectroscopic matches, with {np.sum(qso_mask)} QSOs")
sdss_spec_results

Let's cross-match these QSO labels with our Gaia-SDSS sources so we have a record of what a known QSO looks like:

In [ ]:
sdss_qso_coord = SkyCoord(sdss_spec_results[qso_mask]["ra"], sdss_spec_results[qso_mask]["dec"], unit=u.degree)
sdss_matched_coord = SkyCoord(sdss_matched["ra"], sdss_matched["dec"], unit=u.degree)
idx, d2d, _ = sdss_qso_coord.match_to_catalog_sky(sdss_matched_coord)
close_match = d2d < 1 * u.arcsec  # boolean mask for finding close matches
print(f"Found {np.sum(close_match)} close matches between QSO spectra and Gaia-SDSS sources")

Now create a boolean column for the SDSS sources to denote if they are a QSO

In [ ]:
sdss_matched["known_qso"] = False  # initially set as False for all
sdss_matched["known_qso"][idx[close_match]] = True  # and set to True for those indices with a close match
sdss_matched

## 5. Investigate features of QSOs compared to other field sources to find QSO candidates

We will now look at some properies of our sources, making use of the `"known_qso"` column to investigate what a QSO
"should" look like in these properties.

Firstly, we stack our two tables, `gaia_matched` and `sdss_matched`, in a column-wise manner
([`hstack`](https://docs.astropy.org/en/stable/api/astropy.table.hstack.html#astropy.table.hstack)).
This produces our final `sources` table

In [ ]:
sources = table.hstack([gaia_matched, sdss_matched], table_names=["gaia", "sdss"])
qso_mask = sources["known_qso"]  # boolean mask giving known QSO sources (we can use `~qso_mask` to mean `not qso_mask`)
print(f"There are {len(sources)} sources, with {len(sources[qso_mask])} known QSOs.")
sources.info


### Proper motions

We hypothesised that QSOs could be found as zero proper motion sources within Gaia. We can determine some simple
statistics on proper motions and plot them for known QSOs, and other sources.

In [ ]:
print(f"Proper motions range from {sources[qso_mask]['pmtot'].min():.3f} to {sources[qso_mask]['pmtot'].max():.3f} mas/yr for known QSOs")
print(f"Proper motions range from {sources[~qso_mask]['pmtot'].min():.3f} to {sources[~qso_mask]['pmtot'].max():.3f} mas/yr for other sources")
print(f"The mean proper motion is {sources[qso_mask]['pmtot'].mean():.3f} mas/yr for known QSOs")
print(f"The mean proper motion is {sources[~qso_mask]['pmtot'].mean():.3f} mas/yr for other sources")
print(f"There are {np.sum(sources[qso_mask]['pm_significance'] < 2)} known QSOs with a < 2-sigma proper motion")
print(f"There are {np.sum(sources[~qso_mask]['pm_significance'] < 2)} other sources with a < 2-sigma proper motion")

fig, (ax0, ax1, ax2) = plt.subplots(1,3)

ax0.errorbar(sources[~qso_mask]["pmra"], sources[~qso_mask]["pmdec"], xerr=sources[~qso_mask]["pmra_error"], yerr=sources[~qso_mask]["pmdec_error"], marker="o", markersize=1, ls="", alpha=0.1, label="other sources")
ax0.errorbar(sources[qso_mask]["pmra"], sources[qso_mask]["pmdec"], xerr=sources[qso_mask]["pmra_error"], yerr=sources[qso_mask]["pmdec_error"], marker="d", markersize=3, ls="", label="known QSOs")
ax0.set_xlabel("Proper Motion in RA [mas/yr]")
ax0.set_ylabel("Proper Motion in Dec [mas/yr]")
ax0.legend()

ax1.errorbar(sources[qso_mask]["pmra"], sources[qso_mask]["pmdec"], xerr=sources[qso_mask]["pmra_error"], yerr=sources[qso_mask]["pmdec_error"], marker="d", markersize=3, ls="", label="known QSOs", c="C1")
ax1.set_xlabel("Proper Motion in RA [mas/yr]")
ax1.set_ylabel("Proper Motion in Dec [mas/yr]")
ax1.legend()

ax2.hist(sources[~qso_mask]['pm_significance'], bins=np.arange(41), label="other sources")
ax2.hist(sources[qso_mask]['pm_significance'], bins=np.arange(41), label="known QSOs")
ax2.set_xlabel("Proper motion significance (sigma)")
ax2.set_ylabel("N")
ax2.legend()

It looks like QSOs, as expected, cluster around zero proper motion in Gaia! We also have a number of other
sources which do the same, but are not known QSOs. We want to assess how this subset of sources look in other
properties with respect to QSOs, in order to select from it targets for a hypothetical new QSO-confirmation spectroscopy
proposal.

### Colours

Colours of sources can be revealing of their nature, albeit with significant overlap, particularly in optical colours.


First we need to calculate colour columns for our sources. However, we need to be aware that SDSS can fill in dummy
`-9999.0` values for its photometry, when the data is missing/bad. To avoid this we explicitly set those values to
`np.nan` (i.e. Not A Number) so the are not used in any statistics, or plotting.

In [ ]:
for f in ["u", "g", "r", "i", "z"]:
    sources[f"psfMag_{f}"][sources[f"psfMag_{f}"] == -9999.0] = np.nan

In [ ]:
sources["u_g"] = sources["psfMag_u"] - sources["psfMag_g"]
sources["g_r"] = sources["psfMag_g"] - sources["psfMag_r"]
sources["r_i"] = sources["psfMag_r"] - sources["psfMag_i"]
sources["i_z"] = sources["psfMag_i"] - sources["psfMag_z"]

We also need a make a row mask to give us sources which are consistent with zero proper motion, but are **not** known QSOs:

In [ ]:
zero_pm = sources["pm_significance"] < 2  # a boolean mask giving rows that are consistent with zero proper motion
zero_pm_not_qso_mask = zero_pm & ~qso_mask  # combine with our "not qso" mask
print(f"There are {np.sum(zero_pm_not_qso_mask)} zero proper motion sources not classified as QSOs")

And now we can plot colours for our three source types:
* Known QSOs (`sources[qso_mask]`)
* Sources consistent with zero proper motion that are not known to be QSOs (`sources[zero_pm_not_qso_mask]`)
* Sources consistent with significant proper motion not known to be QSOs (`sources[~zero_pm_not_qso_mask]`)

In [ ]:
fig, axes = plt.subplots(3, 2)
axes = axes.ravel()
colour_columns = [["u_g", "g_r"], ["u_g", "r_i"], ["u_g", "i_z"], ["g_r", "r_i"], ["g_r", "i_z"], ["r_i", "i_z"]]

for i, (colour_column1, colour_column2) in enumerate(colour_columns):
    ax = axes[i]
    ax.scatter(sources[~zero_pm_not_qso_mask][colour_column1], sources[~zero_pm_not_qso_mask][colour_column2], s=5, alpha=0.1, c="C7", label="Non-zero PM, not QSO")
    ax.scatter(sources[zero_pm_not_qso_mask][colour_column1], sources[zero_pm_not_qso_mask][colour_column2], s=10, c="C2", label="Zero PM, not QSO")
    ax.scatter(sources[qso_mask][colour_column1], sources[qso_mask][colour_column2], s=10, c="C1", marker="d", label="Known QSOs")
    ax.set_xlabel(colour_column1)
    ax.set_ylabel(colour_column2)
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 4)
    ax.legend()
plt.tight_layout()


Even by eye we can see that there is good agreement between a significant fraction of our ~zero proper motion sources
and known QSOs in colour-colour space. One can imagine placing some cuts on colour to obtain a good list of QSO
candidates.


In [ ]:
# Write the sources data file here, to be used for cross-matching exercises
# sources.write("data/sources_xm.csv")


-----------

## Example Exercises (cross-matching)

You can pick an example exercise from here for the assessment if you wish. (Or perform your own project-relevant
data-mining, as per the assessment instructions on the MPAGS page.)

Use the link to the csv file in the repo `sources_xm.csv` to import the source data easily into your program of choice.
The sources are the result of this notebook's workings - sections 1 to 4 - using a 1 degree cone centred on RA=180 deg,
Dec=30deg.

### a. Cross match QSO candidates with WISE

Cross-match any sources consistent with zero proper motion that are not known QSOs with
[WISE](https://irsa.ipac.caltech.edu/Missions/wise.html). You can use any method or programme you like - there is a csv
file in the github repo (`data/sources_xm.csv`) to import the data easily into your program of choice. The sources are
from a 1 degree cone centred on RA=180 deg, Dec=30deg.

How many sources get a cross-match with WISE? Try varying your cross-matching radius (`close_match` above) - WISE has
much poorer spatial resolution. What do you see in the WISE photometric colours of the cross-matched sources?
See Section 3 and Figures 3 and 4 of [Heintz et al. 2018](https://arxiv.org/pdf/1805.03394.pdf) for inspiration.

The catalogue within WISE to use is the AllWISE Source Catalogue. You could query it via the online gui
[here](https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-scan?mission=irsa&submit=Select&projshort=WISE) or use
[Astroquery's Irsa](https://astroquery.readthedocs.io/en/latest/irsa/irsa.html) interface, similarly to how the SDSS
interface is used in this notebook. Note to additionally pass `catalog="allwise_p3as_psd"` argument to query the
correct table.

*(At the time of writing - Feb 2021 - the `Irsa.print_catalogs()` functionality is
[broken](https://github.com/astropy/astroquery/issues/2002), but ordinarily you could use this to get a list of all
Irsa resources you can query directly through Astroquery.)*

### b. Check other databases for variability and classification of QSO candidates

QSOs are a subset of [Active Galactic Nuclei](https://en.wikipedia.org/wiki/Active_galactic_nucleus) and they should
typically exhibit variability.

Select zero proper-motion sources that are not known QSOs and apply some colour cuts to get a small subset of QSO
candidates. Check other variability/transient databases (e.g. [Gaia Alerts](http://gsaweb.ast.cam.ac.uk/alerts/home) or
the [Lasair](https://lasair.roe.ac.uk/) or [ALeRCE](https://alerce.online/) ZTF brokers) to try obtain light curves for
any of them. Comment on the variability (or lack of) you find for any of the sources. Search other general data
resources, such as [NED](https://ned.ipac.caltech.edu/) and [SIMBAD](http://simbad.u-strasbg.fr/simbad/), to find out
more information on the objects.

-----------

## 6. Train a Random Forest classifier to classify sources as QSOs based on proper-motion and colour

We will train a (really simple) [Random Forest](https://en.wikipedia.org/wiki/Random_forest) classifier that, for a
given source, will predict a label (the probability of being a QSO) based on a set of features (its SDSS colours
and proper motion).

For this we will need larger numbers of confirmed QSO objects, since we need to show the RF the full extent of what a
QSO can look like in our selected features. If we have only a few, as in the above working, our classifier will
perform poorly.

We will concidely perform a slimmed-down version of the above analysis, here selecting over a wider cone radius and
cross-matching Gaia only to SDSS spectroscopic sources (such that we can robustly label all objects as QSO or not).

In [ ]:
# Define our new, larger cone
ra_cone = 180 * u.degree
dec_cone = 40 * u.degree
coord = SkyCoord(ra=ra_cone, dec=dec_cone, frame='icrs')
region_radius = radius / np.cos(np.deg2rad(coord.dec.degree))  # correction for cos(dec) factor in ra
radius = 4 * u.degree

In [ ]:
# Cone search in Gaia
Gaia.ROW_LIMIT = -1
job = Gaia.cone_search_async(coord, radius)
gaia_results = job.get_results()
gaia_results = gaia_results[(gaia_results["phot_g_mean_mag"] > 16) & (gaia_results["phot_g_mean_mag"] < 20.5) & (gaia_results["astrometric_params_solved"] == 31)]
print(f"Found {len(gaia_results)} Gaia sources")

In [ ]:
# Cone search in SDSS for spectroscopic objects
specobj_fields=["class", "sciencePrimary"]
photoobj_fields=["ra", "dec", "type", "psfMag_u", "psfMag_g", "psfMag_r", "psfMag_i", "psfMag_z"]
sdss_spec_results = SDSS.query_region(coord, region_radius, spectro=True, specobj_fields=specobj_fields, photoobj_fields=photoobj_fields)
sdss_spec_results = sdss_spec_results[(sdss_spec_results["sciencePrimary"] == 1) & (sdss_spec_results["type"] == 6)]
print(f"Found {len(sdss_spec_results)} SDSS spectroscopic sources, with {np.sum(sdss_spec_results['class'] == 'QSO')} QSOs")

In [ ]:
# Cross match Gaia and SDSS and stack into single sources table
gaia_coord = SkyCoord(gaia_results["ra"], gaia_results["dec"], unit=u.degree)
sdss_spec_coord = SkyCoord(sdss_spec_results["ra"], sdss_spec_results["dec"], unit=u.degree)
idx, d2d, _ = gaia_coord.match_to_catalog_sky(sdss_spec_coord)
close_match = d2d < 1 * u.arcsec
gaia_matched = gaia_results[close_match]
sdss_spec_matched = sdss_spec_results[idx[close_match]]
sources = table.hstack([gaia_matched, sdss_spec_matched], table_names=["gaia", "sdss"])
print(f"Found {len(sources)} cross-matches between Gaia and SDSS spectroscopic sources, with {np.sum(sources['class'] == 'QSO')} QSOs")

In [ ]:
# Add feature columns to sources
pmtot = (sources["pmra"]**2 + sources["pmdec"]**2)**0.5
pmtot_error = (sources["pmra_error"]**2 + sources["pmdec_error"]**2 + 2 * sources["pmra_pmdec_corr"] * sources["pmra_error"] * sources["pmdec_error"])**0.5
sources["pm_significance"] = pmtot / pmtot_error
sources["u_g"] = sources["psfMag_u"] - sources["psfMag_g"]
sources["g_r"] = sources["psfMag_g"] - sources["psfMag_r"]
sources["r_i"] = sources["psfMag_r"] - sources["psfMag_i"]
sources["i_z"] = sources["psfMag_i"] - sources["psfMag_z"]


Typically in RFs "labels" (`1` = QSO, `0`= not QSO) are denoted as `y`, with "features" (`pm_significance, u_g, g_r...`)
denoted as `X`.

In [ ]:
y = (sources["class"] == "QSO").astype(np.int)  # convert the boolean condition into 1/0 labels
# slightly awkward way of making our features since we cannot pass an astropy table - the input must have a single dtype
features = ["pm_significance", "u_g", "g_r", "r_i", "i_z", "phot_g_mean_mag"]
X = np.empty((len(y), len(features)))
for i, feat in enumerate(features):
    X[:, i] = sources[feat]

We need to split these into two subsets. We **train** the model on one, and **test** that it performs well on data it
has never seen before. This can help preventing overfitting in the model (i.e. when model memorises its training data
and is not generalisable to the problem at hand).

Scikit learn has a function to do this for us, be default 25% will be reserved for testing.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

Now we train our model on the **train** subset:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0, n_jobs=1, class_weight="balanced_subsample")
model = clf.fit(X_train, y_train)

.. and then test the model with the **test** subset. We calculate some high-level assessment of how well it
performs: defined as the accuracy (balanced for the mis-matched sample sizes). Essentially what
percentage of the time does it correctly label the unseen test data?

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_test, y_test, scoring="balanced_accuracy", cv=10)
print(f"Random Forest gives a {scores.mean()*100:.1f} +/- {scores.std()*100:.1f}% accuracy")

Not bad!

See below for hints on how  to get the model to make a prediction, and also get the probabilities for each class,
for a set of features. See the
[docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) for information
on how to use the Random Forest model further.

In [ ]:
predicted_class = model.predict(X_test)  # remember 0 = not QSO, 1 = QSO
predicted_prob_qso = model.predict_proba(X_test)[:, 1]  # we grab the 2nd column of these class probabilities - the probability of class = 1


In [ ]:
plt.hist(predicted_prob_qso[y_test == 0], histtype="step", label="Not QSO", bins=np.linspace(0,1,40), lw=2)
plt.hist(predicted_prob_qso[y_test == 1], histtype="step", label="QSO", bins=np.linspace(0,1,40), lw=2)
plt.xlabel("Model probability of being QSO")
plt.ylabel("N")
plt.legend()

In [ ]:
# Write the sources data file here, to be used for machine learning exercises
sources.write("data/sources_ml.csv")

-------

## Example Exercises (machine learning)

You can pick an example exercise from here for the assessment if you wish. (Or perform your own project-relevant
data-mining, as per the assessment instructions on the MPAGS page.)

Use the link to the csv file in the repo sources_ml.csv to import the source data easily into your program of choice.
The sources are the result of this notebook's workings - section 5 - using a 4 degree cone centred on RA=180 deg,
Dec=40deg.

### a. Use the model to find new QSOs

Using the trained Random Forest model above, search another (small, ~0.5-1 deg radius) region of sky for Gaia-SDSS cross
matches (getting only photometric sources) and run the model to
[predict](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict)
the labels for your new sources. Manually inspect a few sources with high-probability of being a QSO and find out about
them. Was the model right?

What happens if you try a lower Galactic latitude SDSS-field? *(Be careful, the number of sources returned by searches
over low Galactic latitude regions can shoot up very quickly!)*

### b. Re-train the model using different features

Explore removing or adding features for training the model. What is the impact of the model's accuracy? Are there any
features that do no add any improvement in performance? Are there others in the SDSS or Gaia tables that do improve
the model?

For more advanced investigation, you could look at
[feature selection](https://scikit-learn.org/stable/modules/feature_selection.html) in general for such
feature-based models.